<a href="https://colab.research.google.com/github/agnxsh/task-specific-hf/blob/main/task_specific_hf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Adding a task specific head on top of the BERT model, for dealing with custom NLP tasks

In [109]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###Apart from the last layer, which is the Model head, a layer which is task specific, apart from the other shared layers that are task agnostic, these layers mainly consists of transformer layers and token embeddings.

#### The base model in HuggingFace is usually represented as a python class depicting the BERT class or the GPT-2 class, and what it returns is usually the last hidden layer of the transformer, this is represented in the following way:

output[0] => this is the last hidden state

####In the custom layer for HF, we pass this last hidden state as the input to our new task-specific layer

##Practically speaking the base model has to support the task that we want to add, for example we can't just use DistilBERT for machine translation, because it does not support the same, however DistilBERT can be used for Question-Answering tasks.

In [110]:
# Here we're using a Kaggle Dataset which consists of News Headlines for Sarcasm Detection

In [118]:
!pip install datasets
!pip install transformers
!pip install transformers.modeling_ouputs
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement transformers.modeling_ouputs (from versions: none)
ERROR: No matching distribution found for transformers.modeling_ouputs
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [119]:
import numpy as np
import pandas as pd

from datasets import load_dataset,Dataset,DatasetDict
from transformers import DataCollatorWithPadding, AutoModelForSequenceClassification,Trainer,TrainingArguments,AutoTokenizer, AutoModel,AutoConfig
from transformers.modeling_outputs import TokenClassifierOutput
import torch
import torch.nn as nn
import os
import numpy as np
import pandas as pd
for dirname, _ ,filenames in os.walk("/content/drive/MyDrive/input/"):
  for filename in filenames:
    print(os.path.join(dirname,filename))


/content/drive/MyDrive/input/Sarcasm_Headlines_Dataset_v2.json
/content/drive/MyDrive/input/Sarcasm_Headlines_Dataset.json


In [120]:
dataset_v2_path = "/content/drive/MyDrive/input/Sarcasm_Headlines_Dataset_v2.json"

In [121]:
df = pd.read_json(dataset_v2_path, lines=True)
df.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


#Load Dataset with HF's load_dataset

In [122]:
dataset_hf = load_dataset("json",data_files = dataset_v2_path)

  0%|          | 0/1 [00:00<?, ?it/s]

In [123]:
dataset_hf = dataset_hf.remove_columns(["article_link"])
dataset_hf.set_format('pandas')
dataset_hf=dataset_hf['train'][:]

##Reevaluation of the parameters and labels

In [124]:
dataset_hf.drop_duplicates(subset=['headline'],inplace=True)

dataset_hf=dataset_hf.reset_index()[['headline','label']]

dataset_hf=Dataset.from_pandas(dataset_hf)


# Train Test Valid Split
train_testvalid = dataset_hf.train_test_split(test_size=0.2,seed=15)


test_valid = train_testvalid['test'].train_test_split(test_size=0.5,seed=15)

dataset_hf = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})

dataset_hf

KeyError: ignored

In [ ]:
# #Train Test Valid and Split Data
# train_testvalid = dataset_hf.train_test_split(test_size=0.2,seed=15)

# test_valid=train_testvalid['test'].train_test_split(test_size=0.5,seed=15)

# dataset_hf = DatasetDict({
#     'train':train_testvalid['train']
#     'test':test_valid['test']
#     'valid':test_valid['train']
# })
# dataset_hf